In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.formula.api as smf
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from QBUS2820 import rmse_jack, r2_jack 
from sklearn.linear_model import Ridge 
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
import xgboost as xgb
from sklearn.grid_search import GridSearchCV
from ExtraCode import getResultTable
from sklearn.model_selection import cross_val_score
from QBUS2820 import forward
from sklearn.linear_model import ElasticNet
from QBUS2820 import pcrCV
from QBUS2820 import plsCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [3]:
data = pd.read_csv('Train6_2.csv')
final_train = data.sample(frac=0.6, random_state=1)
final_test = data[data.index.isin(final_train.index)==False]
final_train.head()
y_train = final_train.pop('SalePrice')
y_test = final_test.pop('SalePrice')
y_train = np.log(y_train)



In [4]:
final_train

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Veenker,RoofMatl_CompShg,RoofStyle_Gable,RoofStyle_Hip,RoofStyle_Other,SaleType_WD,SaleType_NonWD
8,1296,0,0,1,2,5,3,1005,0,2,...,0,0,0,0,1,1,0,0,1,0
791,520,600,0,2,2,5,3,0,0,6,...,0,1,0,0,1,1,0,0,1,0
571,864,0,0,1,3,5,3,375,239,0,...,0,0,0,0,1,1,0,0,1,0
224,1107,0,0,1,3,5,3,474,38,0,...,0,0,0,0,1,1,0,0,1,0
498,1187,0,0,1,3,5,3,0,0,6,...,0,0,0,0,1,1,0,0,1,0
262,1179,0,0,1,2,5,1,248,0,2,...,0,0,1,0,1,1,0,0,1,0
73,1040,0,0,1,2,3,3,0,0,4,...,0,0,0,0,1,1,0,0,1,0
795,1636,0,0,1,3,5,3,63,0,2,...,0,1,0,0,1,1,0,0,1,0
670,926,678,0,1,3,5,3,0,0,6,...,0,0,0,0,1,1,0,0,1,0
731,1661,0,0,1,3,5,3,0,0,6,...,0,0,0,0,1,1,0,0,1,0


In [5]:
pred = []
method = []

In [11]:
pcr=pcrCV(final_train, y_train)
predpcr = pcr.predict(final_test)
method.append('PCR')
pred.append(np.exp(predpcr))


In [15]:
neighbor = np.arange(1,101)
optimalk = 0
low = np.inf
for i in neighbor:
    knn1 = KNeighborsRegressor(n_neighbors=i) 
    predictions = knn1.fit(final_train, y_train)
    predictionsfin = predictions.predict(final_test)
    rmse = np.sqrt(mean_squared_error(y_test, np.exp(predictionsfin)))
    if rmse < low:
        low = rmse
        optimalk = i
print('Optimal k is {}'.format(optimalk))

Optimal k is  6


In [18]:
knn1 = KNeighborsRegressor(n_neighbors=optimalk) 
predictions = knn1.fit(final_train, y_train)
predknn = knn1.predict(final_test)
method.append('KNN with optimal {}'.format(optimalk))
pred.append(np.exp(predknn))

### Testing Models

In [6]:
#Forward selection
fwd = forward()
fwd.fit(final_train, y_train)
predforward = fwd.predict(final_test)
method.append('Forward')
pred.append(np.exp(predforward))

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(final_train,y_train)
predFinalBoost = GBoost.predict(final_test)
method.append('GBoost')
pred.append(np.exp(predFinalBoost))

regr = AdaBoostRegressor(loss='linear', learning_rate = 1, n_estimators = 350)
regr = regr.fit(final_train,y_train)
adapred = regr.predict(final_test)
method.append('AdaBoost')
pred.append(np.exp(adapred))

In [7]:
xgdmat = xgb.DMatrix(final_train, y_train) # Create our DMatrix to make XGBoost more efficient
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)
testFinaldmat = xgb.DMatrix(final_test)
xgpred = final_gb.predict(testFinaldmat)
method.append('XGBoost')
pred.append(np.exp(xgpred))

In [8]:
pred2 = (np.exp(xgpred)+np.exp(adapred)+np.exp(predFinalBoost))/3
pred.append(pred2)
method.append('XGBoost, ada, GB')

In [9]:
pred3 = (np.exp(xgpred)+np.exp(predFinalBoost))/2

pred5 = (np.exp(xgpred)+np.exp(adapred))/2
pred.append(pred3)
pred.append(pred5)
method.append('xg GB')
method.append('xg adaboost')

In [10]:
#Extremely Random forest
regr2 = ExtraTreesRegressor(criterion='mae',max_depth=None,min_samples_split=2)
regr2 = regr.fit(final_train,y_train)

predFinalExtRandomForestlad = regr2.predict(final_test)
pred.append(np.exp(predFinalExtRandomForestlad))
method.append('Random Forest')

In [11]:
pred5 = (np.exp(xgpred)+np.exp(predFinalExtRandomForestlad)+np.exp(predFinalBoost))/3
pred.append(pred5)
method.append('XGBoost, random tree, GB')

In [12]:
pred6 = (np.exp(xgpred)+np.exp(predFinalExtRandomForestlad))/2
pred.append(pred6)
method.append('xg Random tree')

In [13]:
pred10 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(predFinalExtRandomForestlad))/3
pred.append(pred10)
method.append('XGBoost, GBoost Random Forest')

In [14]:
#LASSO
lasso = Lasso(alpha = 1)
lasso.fit(final_train, np.ravel(y_train)) 
pred_L = lasso.predict(final_test)
method.append('LASSO')
pred.append(np.exp(pred_L))

In [15]:
alpha = 10
rg = Ridge(alpha = alpha)
rg.fit(final_train, y_train)
y_pred_rg = (rg.predict(final_test))
y_pred_rg

pred.append(np.exp(y_pred_rg))
method.append('Ridge alpha 10')

In [16]:
pred11 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(predFinalExtRandomForestlad)+np.exp(y_pred_rg))/4
pred.append(pred11)
method.append('XGBoost, GBoost Random Forest Ridge')

In [17]:
pred12 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(predFinalExtRandomForestlad)+np.exp(pred_L))/4
pred.append(pred12)
method.append('XGBoost, GBoost Random Forest LASSO')

In [18]:
regr = ElasticNet(random_state=0, alpha = 1)
    
regr.fit(final_train, y_train)    
pred13 = regr.predict(final_test)
pred.append(np.exp(pred13))
method.append('ENET')

In [19]:
pred12 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(predFinalExtRandomForestlad)+np.exp(pred13))/4
pred.append(pred12)
method.append('XGBoost, GBoost Random Forest ENET')

In [20]:
pred13 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(y_pred_rg))/3
pred.append(pred13)
method.append('XGBoost, GBoost Ridge')

In [21]:
pred14 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(y_pred_rg)+np.exp(predforward))/4
pred.append(pred14)
method.append('XGBoost, GBoost Ridge Forward')

In [22]:
pred15 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(y_pred_rg)+np.exp(predforward)+ np.exp(predFinalExtRandomForestlad))/5
pred.append(pred15)
method.append('XGBoost, GBoost Ridge Forward Tree')

In [23]:
pred16 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(y_pred_rg)+np.exp(predforward)+ np.exp(pred_L))/5
pred.append(pred16)
method.append('XGBoost, GBoost Ridge Forward Lasso')

In [24]:
pred15 = (np.exp(xgpred)+np.exp(predFinalBoost)+ np.exp(predFinalExtRandomForestlad))/3
pred.append(pred15)
method.append('XGBoost, GBoost, RandomTree')

In [25]:

pred15 = ((0.3*np.exp(xgpred))+(0.3*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.2*np.exp(predforward)))
pred.append(pred15)
method.append('XGBoost, GBoost Ridge Forward 30 30 20 20')

In [26]:
pred16 = ((0.4*np.exp(xgpred))+(0.4*np.exp(predFinalBoost))+(0.1*np.exp(y_pred_rg))+(0.1*np.exp(predforward)))
pred.append(pred16)
method.append('XGBoost, GBoost Ridge Forward 40 40 10 10')

In [27]:
pred17 = ((0.2*np.exp(xgpred))+(0.2*np.exp(predFinalBoost))+(0.3*np.exp(y_pred_rg))+(0.3*np.exp(predforward)))
pred.append(pred17)
method.append('XGBoost, GBoost Ridge Forward 20 20 30 30')

In [28]:
pred18 = ((0.35*np.exp(xgpred))+(0.35*np.exp(predFinalBoost))+(0.15*np.exp(y_pred_rg))+(0.15*np.exp(predforward)))
pred.append(pred18)
method.append('XGBoost, GBoost Ridge Forward 35 35 15 15')

In [29]:
pred19 = ((0.25*np.exp(xgpred))+(0.35*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.2*np.exp(predforward)))
pred.append(pred19)
method.append('XGBoost, GBoost Ridge Forward 25 35 20 20')

In [30]:
pred20 = ((0.2*np.exp(xgpred))+(0.4*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.2*np.exp(predforward)))
pred.append(pred20)
method.append('XGBoost, GBoost Ridge Forward 20 40 20 20')

In [31]:
pred21 = ((0.2*np.exp(xgpred))+(0.3*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.3*np.exp(predforward)))
pred.append(pred21)
method.append('XGBoost, GBoost Ridge Forward 20 30 20 30')

In [32]:
pred22 = ((0.2*np.exp(xgpred))+(0.3*np.exp(predFinalBoost))+(0.3*np.exp(y_pred_rg))+(0.2*np.exp(predforward)))
pred.append(pred22)
method.append('XGBoost, GBoost Ridge Forward 20 30 30 20')

In [33]:
pred23 = ((0.2*np.exp(xgpred))+(0.2*np.exp(predFinalBoost))+(0.3*np.exp(y_pred_rg))+(0.3*np.exp(predforward)))
pred.append(pred23)
method.append('XGBoost, GBoost Ridge Forward 20 20 30 30')

In [34]:
pred24 = ((0.3*np.exp(xgpred))+(0.5*np.exp(predFinalBoost))+(0.1*np.exp(y_pred_rg))+(0.1*np.exp(predforward)))
pred.append(pred24)
method.append('XGBoost, GBoost Ridge Forward 30 50 10 10')

In [35]:
pred24 = ((0.2*np.exp(xgpred))+(0.2*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.4*np.exp(predforward)))
pred.append(pred24)
method.append('XGBoost, GBoost Ridge Forward 20 20 20 40')

In [36]:
pred25 = ((0.3*np.exp(xgpred))+(0.3*np.exp(predFinalBoost))+(0.3*np.exp(y_pred_rg))+(0.1*np.exp(predforward)))
pred.append(pred25)
method.append('XGBoost, GBoost Ridge Forward 30 30 30 10')

In [37]:
pred26 = ((0.26*np.exp(xgpred))+(0.27*np.exp(predFinalBoost))+(0.27*np.exp(y_pred_rg))+(0.2*np.exp(predforward)))
pred.append(pred26)
method.append('XGBoost, GBoost Ridge Forward 26 27 27 20')

In [38]:
pred27 = ((0.23*np.exp(xgpred))+(0.24*np.exp(predFinalBoost))+(0.24*np.exp(y_pred_rg))+(0.3*np.exp(predforward)))
pred.append(pred27)
method.append('XGBoost, GBoost Ridge Forward 23 24 24 30')

In [40]:
getResultTable(method,pred)

,Test RMSE,SE,Jack R2,SE,MAE,R-square
Forward,17989.154,1191.084,0.934,0.013,12921.807,0.934
GBoost,20953.695,2994.609,0.910,0.017,13951.533,0.910
AdaBoost,26081.224,3625.195,0.861,0.024,18153.271,0.861
XGBoost,20780.239,3098.565,0.912,0.017,13574.624,0.912
"XGBoost, ada, GB",20940.073,3497.291,0.910,0.020,13820.819,0.910
xg GB,20061.499,3157.555,0.918,0.017,13199.957,0.918
xg adaboost,21848.080,3607.373,0.902,0.022,14478.259,0.902
Random Forest,26355.146,3845.359,0.858,0.026,18228.098,0.858
"XGBoost, random tree, GB",20995.256,3585.678,0.910,0.021,13820.333,0.910
xg Random tree,21950.086,3736.180,0.902,0.023,14502.384,0.902


In [39]:
def getResultTable(rows, predictions):
    columns=['Test RMSE', 'SE', 'Jack R2', 'SE', 'MAE', 'R-square']
    results=pd.DataFrame(0.0, columns=columns, index=rows)

    for row,pred in zip(range(0,len(rows)),predictions):
        results.iloc[row,0], results.iloc[row,1] = rmse_jack(y_test, pred)
        results.iloc[row,2], results.iloc[row,3] = (r2_jack(y_test, pred))
        results.iloc[row,4] = mean_absolute_error(y_test, pred)
        results.iloc[row,5] = r2_score(y_test,pred)
    return results.round(3)

## Kaggle Predictions

In [88]:
data = pd.read_csv('Train6_1.csv')
y_price = data.pop('SalePrice')
kaggle = pd.read_csv('Test6_1.csv')
y_price = np.log(y_price)
xgdmat = xgb.DMatrix(data, y_price) # Create our DMatrix to make XGBoost more efficient
testFinaldmat = xgb.DMatrix(kaggle)

In [89]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)
prediction = final_gb.predict(testFinaldmat)

In [90]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(data,y_price)
predFinalBoost = GBoost.predict(kaggle)


In [91]:
alpha = 10
rg = Ridge(alpha = alpha)
rg.fit(data, y_price)
y_pred_rg = (rg.predict(kaggle))

In [92]:
#Forward selection
fwd = forward()
fwd.fit(data, y_price)
predforward = fwd.predict(kaggle)

In [93]:
#Extremely Random forest
regr2 = ExtraTreesRegressor(criterion='mae',max_depth=None,min_samples_split=2)
regr2 = regr2.fit(data,y_price)

predFinalExtRandomForestlad = regr2.predict(kaggle)

In [49]:
finalprediction = ((0.15*np.exp(prediction))+(0.35*np.exp(predFinalBoost))+(0.1*np.exp(y_pred_rg))+ (np.exp(predFinalExtRandomForestlad)*0.05) + (0.35*np.exp(predforward)))

In [50]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction2 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction})
#Saving results into CSV file 
prediction2.to_csv("Day17_2.csv", index=False)

In [51]:
prediction2

,Id,Prediction
0,1,94684.356346
1,2,170665.225010
2,3,131884.430840
3,4,185056.580670
4,5,144309.154443
5,6,367546.991031
6,7,133951.462568
7,8,277186.160803
8,9,117666.139020
9,10,123059.090298


In [56]:
finalprediction = ((0.15*np.exp(prediction))+(0.35*np.exp(predFinalBoost))+(0.15*np.exp(y_pred_rg))+ (0.35*np.exp(predforward)))

In [57]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction3 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction})
#Saving results into CSV file 
prediction3.to_csv("Day17_3.csv", index=False)

In [58]:
prediction3

,Id,Prediction
0,1,95421.541912
1,2,170990.331273
2,3,131316.670032
3,4,184819.439102
4,5,144433.403074
5,6,374844.434367
6,7,133669.362138
7,8,279495.515424
8,9,117715.228489
9,10,123468.655900


In [65]:
finalprediction4 = ((0.15*np.exp(prediction))+(0.35*np.exp(predFinalBoost))+(0.1*np.exp(y_pred_rg))+ (np.exp(predFinalExtRandomForestlad)*0.05) + (0.35*np.exp(predforward)))

In [66]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction4 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction4})
#Saving results into CSV file 
prediction4.to_csv("Day17_4.csv", index=False)

In [67]:
prediction4

,Id,Prediction
0,1,96732.922370
1,2,169971.484124
2,3,132818.383689
3,4,185337.380752
4,5,145494.643516
5,6,364957.762441
6,7,135598.320813
7,8,279875.964013
8,9,117803.220718
9,10,124437.279579


In [68]:
data = pd.read_csv('Train6_1.csv')
y_price = data.pop('SalePrice')
kaggle = pd.read_csv('Test6_1.csv')
y_price = np.log(y_price)
xgdmat = xgb.DMatrix(data, y_price) # Create our DMatrix to make XGBoost more efficient
testFinaldmat = xgb.DMatrix(kaggle)

In [101]:
finalprediction5 = ((0.15*np.exp(prediction))+(0.35*np.exp(predFinalBoost))+(0.15*np.exp(y_pred_rg)) + (0.35*np.exp(predforward)))

In [102]:
finalprediction5

array([  97876.21617689,  162105.09889434,  134746.68247244, ...,
         85145.44067543,  199786.45098341,  213128.49290679])

In [103]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction5 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction5})
#Saving results into CSV file 
prediction5.to_csv("Day17_5.csv", index=False)